In [ ]:
import numpy as np
from obspy import read_events
from obspy import read_inventory
from obspy.clients.fdsn import Client
client_wm = Client("IRIS")
from obspy.taup import TauPyModel
from obspy import Stream, Trace
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from obspy import UTCDateTime
import matplotlib.pyplot as plt
from obspy.geodetics import locations2degrees,degrees2kilometers
from scipy import signal
plt.rc('font', size=10) 
%matplotlib widget
import datetime
from obspy.signal.trigger import recursive_sta_lta,plot_trigger,coincidence_trigger
import pandas as pd
from scipy.stats import linregress

In [ ]:
inv0 = client_wm.get_stations(
    channel="BH*", starttime=UTCDateTime(2020,9,1,0,0,0),endtime=UTCDateTime(2020,10,1,0,0,0),
    network="AK", sta="SCRK,J25K,K27K",
    level="response")
the_station=inv0[0][2]
evid_dum='https://earthquake.usgs.gov/fdsnws/event/1/query?&starttime=2020-09-01&endtime=2020-9-10&latitude=63.98&longitude=-143.99&maxradius=4&minmagnitude=0'
cat1 = read_events(evid_dum)
evid_dum='https://earthquake.usgs.gov/fdsnws/event/1/query?&starttime=2020-09-01&endtime=2020-9-10&latitude=63.98&longitude=-143.99&minradius=4&maxradius=8&minmagnitude=1'
cat2 = read_events(evid_dum)
evid_dum='https://earthquake.usgs.gov/fdsnws/event/1/query?&starttime=2020-09-01&endtime=2020-9-10&latitude=63.98&longitude=-143.99&minradius=8&maxradius=20&minmagnitude=2'
cat3 = read_events(evid_dum)
evid_dum='https://earthquake.usgs.gov/fdsnws/event/1/query?&starttime=2020-09-01&endtime=2020-9-10&latitude=63.98&longitude=-143.99&minradius=20&maxradius=100&minmagnitude=3'
cat4 = read_events(evid_dum)
evid_dum='https://earthquake.usgs.gov/fdsnws/event/1/query?&starttime=2020-09-01&endtime=2020-9-10&latitude=63.98&longitude=-143.99&minradius=100&maxradius=180&minmagnitude=4'
cat5 = read_events(evid_dum)
cat_all=cat1+cat2+cat3+cat4+cat5
channel = "BHZ"
start_time=UTCDateTime(2020,9,1,0,0,0)
end_time=UTCDateTime(2020,9,10,0,0,0)
#st_2day = client_wm.get_waveforms("AK", "SCRK,K27K,J25K", "*", "BHZ", start_time, end_time)
st_10day = client_wm.get_waveforms("AK", "SCRK", "*", "BHZ", start_time, end_time)
EQ_data=pd.read_csv("detections/SCRK_outputs/X_prediction_results.csv")
eq_arv_t=EQ_data['event_start_time']
eq_arv_s_t=EQ_data['s_arrival_time']
st_10day2=st_10day.copy()
st_10day2.merge(method=0,fill_value=0)
st_10day2.detrend()
st_10day2.filter('bandpass',freqmin=0.8, freqmax=25.0)
trace=st_10day2[0]
model = TauPyModel(model="prem")
df = trace.stats.sampling_rate
npts = trace.stats.npts
t = np.arange(npts, dtype=np.float32) / df
cft = recursive_sta_lta(trace.data, int(5 * df), int(10 * df))
eqt_array=[]
for eqtm in eq_arv_t:
    dumt=UTCDateTime(eqtm)
    #ax.vlines(dumt-trace.stats.starttime, ii,jj, color='y',linestyles='dashdot', lw=3)
    eqt_array.append(dumt-trace.stats.starttime)
    picker_time_a=[]
for event in cat_all:
    disD=locations2degrees(the_station.latitude,the_station.longitude,event.origins[0].latitude,event.origins[0].longitude)
    if event.origins[0].depth<0:
        event.origins[0].depth=0
    arrivals = model.get_travel_times(source_depth_in_km=event.origins[0].depth/1000,
                                  distance_in_degree=disD)    
    picker_time_a.append(event.origins[0].time+arrivals[0].time-trace.stats.starttime)

In [ ]:
max_search_range_b=10
max_search_range_a=30
cft_thr=1.35

sel_cat_list=[]

lij=-1
for event in cat_all:
    lij=lij+1
    disD=locations2degrees(the_station.latitude,the_station.longitude,event.origins[0].latitude,event.origins[0].longitude)
    picker_time=picker_time_a[lij]
    t0=int(picker_time/trace.stats.delta)
    tarray=range(int(t0-max_search_range_b*trace.stats.sampling_rate),int(t0+max_search_range_a*trace.stats.sampling_rate))
    try:
        if np.max(cft[tarray])>cft_thr:
            sel_cat_list.append(event)
            # if flag==0:
            #     cat_all_sel=event
            #     flag=1
            # else:
            #     cat_all_sel=cat_all_sel+event
    except Exception:
        continue
    
for i,event in enumerate(sel_cat_list):
    if i==0:
        dum=cat_all[0:1]
        dum.clear()
        all_sel_event=dum+event
    else:
        all_sel_event=all_sel_event+event
lij=-1
eqt_max_time=25
for i,picker_time in enumerate(picker_time_a):
    if np.abs(np.array(eqt_array)-picker_time).min()<eqt_max_time:
        lij=lij+1
        if lij==0:
            dum=cat_all[0:1]
            dum.clear()
            eqt_sel_event=dum+cat_all[i]
        else:
            eqt_sel_event=eqt_sel_event+cat_all[i]
eqtpall_sel_event=all_sel_event+eqt_sel_event
event_in_list=[]
for event in eqtpall_sel_event:
    disD=locations2degrees(the_station.latitude,the_station.longitude,event.origins[0].latitude,event.origins[0].longitude)
    event_list=[event.origins[0].longitude,event.origins[0].latitude,event.origins[0].depth,event.magnitudes[0].mag,disD,event.event_type,event.origins[0].time]
    event_in_list.append(event_list)
eqtpall_sel_event_pd=pd.DataFrame(event_in_list,columns=['longitude','latitude','depth','magnitude','disD','event_type','time'])
bh_catalog=eqtpall_sel_event_pd.drop_duplicates(subset=['longitude','latitude','magnitude'])

In [ ]:
bh_catalog

In [ ]:
def threshold_trigger(th,record_time,trace):
    record_number=record_time*trace.stats.sampling_rate
    abs_st_data=np.abs(trace.data)
    ind_th=[]
    recording=False
    for i,abs_st in enumerate(abs_st_data):
        if abs_st>th and recording is False:
            ind_th.append(i)
            recording=True
        if ind_th!=[] and (i-ind_th[-1])>record_number:
            recording=False
    return len(ind_th),ind_th

def cal_bhcat_t(bh_catalog):
    cat_arl_t=[]
    cat_arl_i=[]
    for i in bh_catalog.index:
        arrivals = model.get_travel_times(source_depth_in_km=bh_catalog.loc[i,'depth']/1000,
                                    distance_in_degree=bh_catalog.loc[i,'disD'])    
        picker_time=bh_catalog.loc[i,'time']+arrivals[0].time-trace.stats.starttime
        cat_arl_t.append(picker_time)
        cat_arl_i.append(i)
    return cat_arl_t,cat_arl_i


def event_associate(pre_time,after_time,cat_arl_t,t,ind_th,cat_arl_i):
    n_recorded=0
    index_triggered=[]
    index_cat=[]
    for i in ind_th:
        for j,cat_t in enumerate(cat_arl_t):
            if cat_t>t[i]-pre_time and cat_t<t[i]+after_time:
                n_recorded=n_recorded+1
                index_triggered.append(cat_arl_i[j])
                index_cat.append(j)
                break
    return n_recorded,index_triggered,index_cat
def threshold_trigger_w_avg(th_fold,record_time,trace,window_len):
    record_number=record_time*trace.stats.sampling_rate
    abs_st_data=np.abs(trace.data)
    ind_th=[]
    recording=False
    rtt=0
    cm_sum=0
    ltavg=np.sum(abs_st_data[0:window_len])/window_len
    for i,abs_st in enumerate(abs_st_data):
        rtt=rtt+1
        if rtt>window_len:
            rtt=0
            ltavg=cm_sum/window_len
            #print(ltavg)
            cm_sum=abs_st
        cm_sum=cm_sum+abs_st
        if abs_st>th_fold*ltavg and recording is False:
            ind_th.append(i)
            recording=True
        if ind_th!=[] and (i-ind_th[-1])>record_number:
            recording=False
    return len(ind_th),ind_th
def cal_tauc(tr,sat_t,end_t):
    df = tr.stats.sampling_rate
    npts = tr.stats.npts
    t = np.arange(npts, dtype=np.float32) / df
    ind_range=(t<end_t) * (t>sat_t)
    r=np.sum(np.diff(tr[ind_range])**2/tr.stats.delta)/np.sum(tr[ind_range]**2)
    return 2*np.pi/np.sqrt(r)

In [ ]:
bh_catalog_nd=bh_catalog.reset_index()
#trace=st_10day2[0]
event_array=[]
event_arlt_array=[]
pre_min=0.5
after_min=5
for i in range(len(bh_catalog_nd)):

    event=bh_catalog_nd.loc[i]
    arrivals = model.get_travel_times(source_depth_in_km=event.depth/1000,distance_in_degree=event.disD)  
    arl_time=event.time+arrivals[0].time
    #arl_time=event_arlt_array[i]
    trace_slc=trace.slice(arl_time-pre_min*60,arl_time+after_min*60)
    event_array.append(trace_slc)
    event_arlt_array.append(arl_time)
pre_time=80
after_time=10
th_fold=10
th_value=100
window_len=50*60*1
rest_time=30*60
event_sel_array=[]
event_arlt_sel_array=[]
ind_slt=[]
for i in range(len(bh_catalog_nd)):
#for i in range(1):
    event=bh_catalog_nd.loc[i]
    tr=event_array[i]
    df = tr.stats.sampling_rate
    npts = tr.stats.npts
    t = np.arange(npts, dtype=np.float32) / df
    arl_time=event_arlt_array[i]
    #n,ind_th=threshold_trigger_w_avg(th_fold,rest_time,tr,window_len)
    n,ind_th=threshold_trigger(th_value,rest_time,tr)
    for k in ind_th:
        if arl_time-tr.stats.starttime>t[k]-pre_time and arl_time-tr.stats.starttime<t[k]+after_time:
            event_sel_array.append(tr)
            event_arlt_sel_array.append(arl_time)
            ind_slt.append(i)

In [ ]:
# for detected events
sat_t=20
end_t=45
tauc_over_events=[]
mag_array=np.array(bh_catalog_nd.loc[ind_slt]['magnitude'])
for i,trc in enumerate(event_sel_array):
    #event=bh_cat_sort_mag_sel.loc[i]
    tauc_over_events.append(cal_tauc(trc,sat_t,end_t))
A = np.vstack([mag_array, np.ones(len(mag_array))]).T
logtauc=np.log(tauc_over_events)
m, c = np.linalg.lstsq(A, logtauc, rcond=None)[0]

In [ ]:
len(ind_slt)

In [ ]:
# for all events
sat_t=20
end_t=45
tauc_over_events=[]
mag_array=np.array(bh_catalog_nd['magnitude'])
for i,trc in enumerate(event_array):
    #event=bh_cat_sort_mag_sel.loc[i]
    tauc_over_events.append(cal_tauc(trc,sat_t,end_t))
A = np.vstack([mag_array, np.ones(len(mag_array))]).T
logtauc=np.log(tauc_over_events)
m, c = np.linalg.lstsq(A, logtauc, rcond=None)[0]

In [ ]:
y=m*mag_array+c
msft=np.sqrt(np.sum((y-mag_array)**2))
convar=np.sum((mag_array-mag_array.mean())*(logtauc-np.array(logtauc).mean()))/mag_array.size
plt.figure(figsize=(8,6))
plt.scatter(mag_array,logtauc,label='Original tauc')
plt.plot(mag_array,y,'b-',label='fitted tauc')
plt.xlabel('magnitude (mw)')
plt.ylabel('log(tauc)')
#plt.title("misfit= %f, m= %f, cov= %f" %(msft,m,convar))
corr_mat=np.corrcoef(mag_array,logtauc)
plt.title('correlation coefficient = %4.3f' %(corr_mat[0][1]))

In [ ]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w
def cal_spec_from_trc(trc,wid):
    dt=trc.stats.delta
    N=trc.stats.npts
    T=N*dt
    dw=2*np.pi/T
    w=np.array(range(int(N/2)))*dw
    spec_amp=np.abs(np.fft.fft(trc.data)[:int(N/2)])
    spec_smth=moving_average(spec_amp,wid)
    #max_ind=spec_amp.argmax()
    max_ind=spec_smth.argmax()
    if wid==1:
        w_plot=w
    else:
        w_plot=w[:-wid+1]
    return spec_smth, w_plot[max_ind],w_plot

In [ ]:
tauc_test_peakf=[]
win_len=1200
peak_f_a=[]
for i,trc in enumerate(event_sel_array):
    #event=bh_cat_sort_mag_sel.loc[i]
    tauc_test_peakf.append(cal_tauc(trc,sat_t,end_t))
    spec_amp,peak_w,w_plot=cal_spec_from_trc(trc,win_len)
    peak_f_a.append(peak_w/(2*np.pi))

In [ ]:
tauc_test_peakf=[]
peak_f_a=[]
win_len=100
for i,trc in enumerate(event_sel_array):
    #event=bh_cat_sort_mag_sel.loc[i]
    #tauc_test_peakf.append(cal_tauc(trc,sat_t,end_t))
    spec_amp,peak_w,w_plot=cal_spec_from_trc(trc,win_len)
    peak_f_a.append(peak_w/(2*np.pi))

In [ ]:
plt.figure(figsize=(8,6))
peak_inv=1/np.array(peak_f_a)
x=mag_array
A = np.vstack([x, np.ones(len(x))]).T
#logtaucf=np.log(tauc_test_peakf)
#logtaucf=mag_array
m, c = np.linalg.lstsq(A, peak_inv, rcond=None)[0]
y=m*x+c
plt.scatter(x,peak_inv)
plt.plot(x,y,'b-')
plt.ylabel('1/(peak frequency) (s)')
plt.xlabel('magnitude')
#plt.title('mag vs 1/f (freq peak from 25s segment)')
corr_mat=np.corrcoef(x,peak_inv)
plt.title('correlation coefficient = %4.3f' %(corr_mat[0][1]))

In [ ]:
st_10day_unfilter=st_10day.copy()
st_10day_unfilter_1=st_10day[0:3].merge(method=0,fill_value=0).detrend()
st_10day_unfilter_2=st_10day[3:5].merge(method=0,fill_value=0).detrend()
st_10day_unfilter_3=st_10day[5:8].merge(method=0,fill_value=0).detrend()
st_10day_unfilter=st_10day_unfilter_1+st_10day_unfilter_2+st_10day_unfilter_3
st_10day_unfilter[2].data=np.append(st_10day_unfilter[2].data,0)
# get unfilterd data slice
trace=st_10day_unfilter[2]
df = trace.stats.sampling_rate
npts = trace.stats.npts
t = np.arange(npts, dtype=np.float32) / df
T=npts/df
dw=2*np.pi/T
event_array_unflt=[]
event_arlt_array_unflt=[]
pre_min=0.3
after_min=2
cat_bh_uft=bh_catalog_nd.loc[ind_slt].reset_index()
for i in range(len(cat_bh_uft)):
    event=cat_bh_uft.loc[i]
    arrivals = model.get_travel_times(source_depth_in_km=event.depth/1000,distance_in_degree=event.disD)  
    arl_time=event.time+arrivals[0].time
    trace_slc=trace.slice(arl_time-pre_min*60,arl_time+after_min*60)
    event_array_unflt.append(trace_slc)
    event_arlt_array_unflt.append(arl_time)
event_arlt_array_unflt_h0p1=[]
for i in range(len(cat_bh_uft)):
    flt_trc=event_array_unflt[i].detrend().filter('bandpass', freqmin=0.1,freqmax=25)
    event_arlt_array_unflt_h0p1.append(flt_trc)

In [ ]:
win_len=80
f_divide=1
f_highcut=5
ratio_freq=[]
for i in range(len(cat_bh_uft)):
    trc=event_arlt_array_unflt_h0p1[i]
    dt=trc.stats.delta
    N=trc.stats.npts
    T=N*dt
    dw=2*np.pi/T
    w=np.array(range(int(N/2)))*dw
    spec_plot,peak_w,w_plot=cal_spec_from_trc(trc,win_len)
    w_sel_a=(w_plot>f_divide*2*np.pi)*(w_plot<f_highcut*2*np.pi)
    w_sel_an=w_plot<=f_divide*2*np.pi
    ratio_freq.append(spec_plot[w_sel_an].sum()/spec_plot[w_sel_a].sum())

In [ ]:
plt.figure(figsize=(8,6))
x=mag_array
log_ratio=np.log(ratio_freq)
A = np.vstack([x, np.ones(len(x))]).T
#logtaucf=np.log(tauc_test_peakf)
#logtaucf=mag_array
m, c = np.linalg.lstsq(A, log_ratio, rcond=None)[0]
y=m*x+c
plt.plot(x,y,'b-')
plt.scatter(x,log_ratio)
plt.xlabel('magnitude (mw)')
plt.ylabel('log(low/high freq content)')
corr_mat=np.corrcoef(x,log_ratio)
plt.title('correlation coefficient = %4.3f' %(corr_mat[0][1]))

In [ ]:
testtrcn=5
test_trc=event_arlt_array_unflt_h0p1[testtrcn].copy()
test_trc.detrend().plot()
test_trc=event_arlt_array_unflt_h0p1[testtrcn].copy()
test_trc.detrend().filter('bandpass',freqmin=0.1, freqmax=24).plot()
test_trc=event_arlt_array_unflt_h0p1[testtrcn].copy()
test_trc.detrend().filter('bandpass',freqmin=2, freqmax=8).plot()
test_trc=event_arlt_array_unflt_h0p1[testtrcn].copy()
test_trc.detrend().filter('bandpass',freqmin=0.8, freqmax=2).plot()
test_trc=event_arlt_array_unflt_h0p1[testtrcn].copy()
test_trc.detrend().filter('bandpass',freqmin=0.8, freqmax=2).plot()


In [ ]:
test_trc=event_arlt_array_unflt_h0p1[testtrcn].copy()
test_trc.detrend().filter('bandpass',freqmin=2, freqmax=4).plot()
test_trc=event_arlt_array_unflt_h0p1[testtrcn].copy()
test_trc.detrend().filter('bandpass',freqmin=2, freqmax=20).plot()